In [12]:
import pandas as pd
import numpy as np

In [13]:
train = pd.read_csv("/Users/polyanaboss/Downloads/house_prices_kaggle/train.csv")
val = pd.read_csv("/Users/polyanaboss/Downloads/house_prices_kaggle/test.csv")

**Ход работы**:

1) соединяем оба датасета, чтобы импьютинг и дамми переменные не создавали разное количество колонок(при раздельном импьютинге)

2) получаем готовый датасет для передачи в модели

3) пробуем 2 регрессии - Lasso and Ridge

4) используем grid search и RidgeCV для оптимизации второго алгоритма

5) пробуем более мощные алгоритмы - ансамбль LIGHTGBM - градиентый бустинг, только более продвинутый(из отдельной библиотеки)

6) выбираем лучшую модель на основе ошибки на тестовой выборке

7) делаем предсказания на валидационном сете

P.S: местами комментарии английские, так как русских аналогов терминов либо нет, либо они очень неудобные, спасибо за понимание

# Imputing 

In [14]:
#отделяем таргет от всего датасета
target = train['SalePrice']
train.drop('SalePrice', axis = 1, inplace = True)

In [16]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [37]:
train.tail()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal
1459,1460,20,RL,75.0,9937,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2008,WD,Normal


In [23]:
#соединяем данные для импьютинга
df = pd.concat((train, val), axis = 0)
#categorical features for imputing
categorical_features = [feature for feature in df.columns if df[feature].dtypes=='O' 
                                       and df[feature].isnull().sum() > 0]
print(len(categorical_features))

23


In [24]:
#num
numerical_features = [feature for feature in df.columns if df[feature].dtypes=='int64' or df[feature].dtypes=='float64']
numerical_features = [feature for feature in numerical_features if df[feature].isnull().sum() > 0]
print(len(numerical_features))



11


In [25]:
from sklearn.impute import SimpleImputer

In [26]:
imputer_cat = SimpleImputer(strategy='most_frequent')
imputer_num = SimpleImputer(strategy='mean')

In [32]:
#imputing for numeric features with most frequent strategy
for feature in categorical_features:
    df[feature + "_imputed"] = imputer_cat.fit_transform(np.array(df[feature]).reshape(-1, 1))
    df.drop(feature, axis = 1, inplace = True)

In [33]:
#imputing for numeric features with mean strategy
for feature in numerical_features:
    df[feature + "_imputed"] = imputer_num.fit_transform(np.array(df[feature]).reshape(-1, 1))
    df.drop(feature, axis = 1, inplace = True)

In [36]:
#number of missing values
(df.isnull().sum() > 0).sum()

0

In [40]:
#getting dummies
df = pd.get_dummies(df, drop_first=True)

In [42]:
#splitting back to train and validation 
train_imp = df.iloc[:1460, :]
val_imp = df.iloc[1460:, :]

In [43]:
val_imp.shape

(1459, 246)

In [46]:
#логарифмируем таргет
target_log = np.log(target)

# Modelling

In [58]:
#linear regression - Lasso and Ridge
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV

In [48]:
#train test split
from sklearn.model_selection import train_test_split

In [49]:
X_train, X_test, y_train, y_test = train_test_split(train_imp, target_log, test_size = 0.2, random_state=0)

In [51]:
#lasso model with defalut params
lasso = Lasso().fit(X_train, y_train)

In [52]:
#making predictions
pred_lasso = lasso.predict(X_test)

In [53]:
#ridge model with defalut params
ridge = Ridge().fit(X_train, y_train)

In [54]:
pred_ridge = ridge.predict(X_test)

In [55]:
#metrics
from sklearn.metrics import mean_squared_error

In [56]:
#rmse for predictions
mean_squared_error(pred_lasso, y_test, squared=False)

0.2654057641886514

In [57]:
mean_squared_error(pred_ridge, y_test, squared=False)

0.2043671077821448

Так как вторая модель показала меньшую ошибку, попробуем потьюнить именно ее

# Grid Search 

Сначала попробуем универсальный метол для всех моделей, затем CV именно для Ridge regression

In [62]:
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid
grid = dict()
grid['alpha'] = np.arange(0, 1, 0.01)
# define search
search = GridSearchCV(ridge, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1).fit(X_train, y_train)

In [64]:
mean_squared_error(search.predict(X_test), y_test, squared=False)

0.20450055068858983

In [66]:
search1 = RidgeCV(alphas=np.arange(0, 1, 0.01), cv = cv).fit(X_train, y_train)

In [67]:
mean_squared_error(search1.predict(X_test), y_test, squared=False)

0.20450055068858983

Видим, что особых плодов это не принесло, попробуем модели посложнее и поумнее, будем использовать **LIGHTGBM**, прокаченную версию градиентного бустинга.

# LightGBM

In [68]:
import lightgbm as lgb

In [69]:
#для этой библиотеки нужны данные, преобразованные специальным методом Dataset
train_data = lgb.Dataset(X_train, label=y_train)
                                        
test_data = lgb.Dataset(X_test, label=y_test)

In [76]:
#некоторые параметры модели, значения взял частично из своего другого проекта, решил попробовать сразу их
defaulf_lgbm_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'regression',
                    'metric': 'mean_squared_error',
                    'subsample': 0.8,
                    'subsample_freq': 1,
                    'learning_rate': 0.005,
                    'num_leaves': 2**8,
                    'min_data_in_leaf': 2**5-1,
                    'feature_fraction': 0.8,
                    'n_estimators': 13000,
                    'extra_trees': True,
                    'boost_from_average': False,
                    #'early_stopping_rounds': 200,
                    'verbose': -1,
                    'seed': 42                }

In [77]:
#fitting the model
lgb_model = lgb.train(defaulf_lgbm_params, train_data, 
                                            valid_sets = [train_data, test_data],
                                            verbose_eval = 100,
                                            num_boost_round = 500
                                     )

[100]	training's l2: 53.1324	valid_1's l2: 53.1783
[200]	training's l2: 19.5143	valid_1's l2: 19.55
[300]	training's l2: 7.17438	valid_1's l2: 7.19872
[400]	training's l2: 2.64423	valid_1's l2: 2.66207
[500]	training's l2: 0.98045	valid_1's l2: 0.993753
[600]	training's l2: 0.368924	valid_1's l2: 0.380146
[700]	training's l2: 0.143589	valid_1's l2: 0.153188
[800]	training's l2: 0.0602237	valid_1's l2: 0.0687718
[900]	training's l2: 0.0291274	valid_1's l2: 0.037169
[1000]	training's l2: 0.0173076	valid_1's l2: 0.025134
[1100]	training's l2: 0.012615	valid_1's l2: 0.0204476
[1200]	training's l2: 0.0105626	valid_1's l2: 0.0184866
[1300]	training's l2: 0.00952415	valid_1's l2: 0.0176145
[1400]	training's l2: 0.00888504	valid_1's l2: 0.0171536
[1500]	training's l2: 0.0084076	valid_1's l2: 0.0168936
[1600]	training's l2: 0.00802574	valid_1's l2: 0.0167501
[1700]	training's l2: 0.00768513	valid_1's l2: 0.0166434
[1800]	training's l2: 0.0073769	valid_1's l2: 0.0165363
[1900]	training's l2: 0.0

In [78]:
mean_squared_error(lgb_model.predict(X_test), y_test, squared=False)

0.12641773387834096

# Submission 

In [80]:
#prediction for validation set
sub = lgb_model.predict(val_imp)

In [83]:
#transforming back from log
sub_normal = np.exp(sub)

In [88]:
final = pd.DataFrame({'Id' : val_imp.Id, 
                 'SalePrice' : sub_normal})

In [90]:
final.to_csv('final_pred.csv', index=False)